## Stanford Algo, Part 1. Programming Assignment 4. 

### 0. Kosaraju algorithm - Computing strongly connected components

Download the following text file (right click and select "Save As..."): SCC.txt

The file contains the edges of a directed graph. Vertices are labeled as positive integers from 1 to 875714. Every row indicates an edge, the vertex label in first column is the tail and the vertex label in second column is the head (recall the graph is directed, and the edges are directed from the first column vertex to the second column vertex). So for example, the  row looks liks : "2 47646". This just means that the vertex with label 2 has an outgoing edge to the vertex with label 47646

Your task is to code up the algorithm from the video lectures for computing strongly connected components (SCCs), and to run this algorithm on the given graph.

Enter the sizes of the 5 largest SCCs in the given graph using the fields below, in decreasing order of sizes. So if your algorithm computes the sizes of the five largest SCCs to be 500, 400, 300, 200 and 100, enter 500 in the first field, 400 in the second, 300 in the third, and so on. If your algorithm finds less than 5 SCCs, then enter 0 for the remaining fields. Thus, if your algorithm computes only 3 SCCs whose sizes are 400, 300, and 100, then you enter 400, 300, and 100 in the first, second, and third fields, respectively, and 0 in the remaining 2 fields.

WARNING: This is the most challenging programming assignment of the course. Because of the size of the graph you may have to manage memory carefully. The best way to do this depends on your programming language and environment, and we strongly suggest that you exchange tips for doing this on the discussion forums.

In [1]:
# Import necessary modules
from time import time
from collections import defaultdict

### 1. Define the function to build the direct and reversed Graphs from the text file

In [2]:
def buildGraph(filename):
        ''' Read input file and build G and G_rev with all arcs reversed'''
        G, G_rev = defaultdict(list), defaultdict(list)
        # Populate dictionary with information from file
        with open(filename) as f:
            for line in f:
                tail, head = line.split()
                tail, head = int(tail), int(head)
                G[tail].append(head)
                G_rev[head].append(tail)
        return G, G_rev

### 2. Define a class of methods and attributes to implement the Kosaraju's Algo

In [3]:
class Kosaraju(object):
    def __init__(self, n):
        self.n = n             # total numver of vertices
        self.explored = set()  # all vertices initialized as unexplored
        self.f = []            # list to store the 'magical' ordering of vertices
        self.s = None          # current leader
        self.leader = defaultdict(int)   # dict of leaders for each vertex          
         
    def firstPass(self, G):
        '''Topo-sort algo for the 1st DFS loop on G_rev'''
        self.explored.clear()  # all verices unexplored
        for i in range(self.n, 0, -1):   # traverse all vertices
            if i not in self.explored:
                S = [i]        # intitialize a stack data structure
                while S:
                    v = S.pop()
                    if v not in self.explored:
                        self.explored.add(v)
                        S.append(v)   # add vertex back to S before neighbors
                        for w in G[v]:
                            if w not in self.explored:
                                S.append(w)
                    else:
                        self.f.append(v)  # add v to the ordering stack on 2nd view
            
    def secondPass(self, G):
        '''Traverse vertices in the reversed order of f and label
            them with vertex from which they are explored'''
        self.explored.clear()   # all vertices unexplored
        while self.f:
            i = self.f.pop()  # traverse f in reverse order
            if i not in self.explored:
                self.s = i    # set current leader
                S = [i]       # initialize a stack data structure
                while S:
                    v = S.pop()
                    if v not in self.explored:
                        self.explored.add(v)
                        self.leader[v] = self.s
                        for w in G[v]:
                            S.append(w)

### 3. Apply the Kosaraju's algo to compute SCCs in our Graph

In [4]:
start = time()

G, G_rev = buildGraph("SCC.txt")       # build the Graph and its reversed version     
V = set(G.keys()).union(G_rev.keys())  # set of the Graph vertices
n = len(V)                             # number of verices in the Graph

finish = time()
print(f'Graphs built in {finish-start:.2f} secs')

Graphs built in 7.17 secs


In [5]:
start = time()

g = Kosaraju(n)     # define an instance of Kasaraju class
g.firstPass(G_rev)  # 1st DFS pass computes 'magical' ordering of vertices
g.secondPass(G)     # 2nd DFS pass finds SCCs in reverse topological order

finish = time()
print(f'Two-pass Kosaruju algo finished in {finish-start:.2f} secs')

Two-pass Kosaruju algo finished in 4.71 secs


In [6]:
# Compute sizes of SCC:
sizeSCC = defaultdict(int)        # define a dict for SCC sizes
leaders = list(g.leader.values()) # list of leader labels for vertices
while leaders:
    v = leaders.pop()             # travers the list of leader labels
    sizeSCC[v] += 1               # count unique values

# Print sizes of the largest SCCs:
largestSCC = sorted([v for v in sizeSCC.values()], reverse=True)[:5]
#print(f'The five largest SCCs: {largestSCC}')